In [4]:
import math
import numpy as np
from numba import jit
from numba import njit
from numba import prange
from scipy.optimize import fsolve

# PRECOMPUTATION

Precomputation is everything that would be part of every heuristik anyway.
This means data that would have to be accessed

**This notebook is used to investigate several computational variants for precomputation**

# 0) Replicate solutions of paper
This goal of this notebook is to understand and replicate the velocity calculation.

This will be one of the key components of the master thesis and impact the heuristical performance to a great deal.

## 0.1 Replicate velocity calculation with understandable formula

In [2]:
def forces_logical(velocity, mass, slope):
    """
    Test if force calculation is correct
    Calculate the travel times in understandable way
    Returns:

    """
    # normalize velocity to ms
    velocity_norm = velocity/3.6
    
    # calculate the 3 main resisting forces
    air_resistance = (1.18*1.18*0.83)/2*(velocity_norm**2)
    rolling_resistance = (0.01*mass*9.81*np.cos(np.arctan(slope)))
    gravity = mass*9.81*np.sin(np.arctan(slope))

    # calculate teh final power
    power = (air_resistance+rolling_resistance+gravity)*velocity_norm/0.95
    return air_resistance, rolling_resistance, gravity, power


Lets check if we can replicate the force values from the paper

In [5]:
power = 340
velocities = [15, 20, 25, 30]
masses = [100, 150, 200, 250]
slopes = [0, 0.02, 0.05, 0.08, 0.1]

print(forces_logical(15, 150, 0.05))
print(forces_logical(25, 100, 0.05))

NameError: name 'forces_logical' is not defined

The slight deviations are because of an error by the author of the original paper

# 1 Speed calculation

This section shows to ways to calculate the speed. One exact way and one optimized for speed.

Both methods would probably be useable for precomputation

## 1.1) Exact speed calculation

In [17]:
POWER = 350
AIR_RESISTANCE_CONSTANT = (1.2225*1.18*0.83)/2/0.95

@njit(fastmath=True)
def velocity_function(velocity, mass, slope, power):
    norm_velocity = velocity/3.6
    rolling_resistance = (0.01*mass*9.81*np.cos(np.arctan(slope)))
    gravity = mass*9.81*np.sin(np.arctan(slope))

    return AIR_RESISTANCE_CONSTANT*(norm_velocity)**3+rolling_resistance*(norm_velocity/0.95)+gravity*(norm_velocity/0.95)-power

def calculate_velocity_precicse(mass, slope):
    velocity_result = fsolve(func=velocity_function, x0=20, args=(mass, slope, POWER), full_output=0)
    if velocity_result > 25:
        return 25
    else:
        return velocity_result

In [18]:
%timeit -n 5000 calculate_velocity_precicse(340, 0)

41.2 µs ± 21.6 µs per loop (mean ± std. dev. of 7 runs, 5000 loops each)


## 1.2) Specialized speed computation

In [48]:
POWER = 350

DRAG_COEFFICIENT = 1.18
RIDER_SURFACE = 0.83
RHO = 1.225
COEFFICIENT_ROLLING = 0.01

AIR_RESISTANCE_CONSTANT = (1.225*1.18*0.83)/2/0.95

@njit(fastmath=True)
def calculate_velocity_fast(mass, slope, accuracy=1):
    """
    We know that There must be a certain limit to the speed and will never drop below 0.  
    This property can be used to calculate the velocity.  

    As this specialized function can be transformed into njit  
    Interestingly enough this method is slower than **scipy.optimize import fsolve** if we don't use the numba compiler.  
    
    Args:
        mass:   The mass of the vehicle
        slope:  The slope of the street on which the driver must drive
        
    """
    rolling_resistance = (COEFFICIENT_ROLLING*mass*9.81*np.cos(np.arctan(slope)))
    gravity = mass*9.81*np.sin(np.arctan(slope))

    # calculate the velocity
    velocity = 0
    while True:
        if AIR_RESISTANCE_CONSTANT*velocity**3+rolling_resistance*(velocity/0.95)+gravity*(velocity/0.95)-POWER < 0:
            velocity += accuracy
        else:
            break
            
    velocity = velocity*3.6
    if velocity > 25:
        return 25
    else:
        return velocity


In [49]:
%timeit calculate_velocity_fast(340, 0, accuracy=0.01)

999 ns ± 22 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [70]:
calculate_velocity_fast(340, -0.002)

23.219999999999665

In [63]:
@njit(fastmath=True)
def calculate_velocity_fast(mass, slope, accuracy=0.01):
    """
    We know that There must be a certain limit to the speed and will never drop below 0.
    This property can be used to calculate the velocity.

    As this specialized function can be transformed into njit
    Interestingly enough this method is slower than **scipy.optimize import fsolve** if we don't use the numba compiler.

    Args:
        mass:   The mass of the vehicle
        slope:  The slope of the street on which the driver must drive

    """
    rolling_resistance = (COEFFICIENT_ROLLING * mass * 9.81 * np.cos(np.arctan(slope)))
    gravity = mass * 9.81 * np.sin(np.arctan(slope))

    # calculate the velocity
    velocity = 0 # start at 25 because that is the max speed!
    while True:
        if AIR_RESISTANCE_CONSTANT*velocity**3 + rolling_resistance*(velocity/0.95)+gravity*(velocity/0.95)-POWER < 0:
            velocity += accuracy
        else:
            break

    velocity = velocity*3.6
    if velocity > 25:
        return 25
    else:
        return velocity

In [64]:
%timeit calculate_velocity_fast(340, 0, accuracy=0.01)

987 ns ± 10.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [65]:
calculate_velocity_fast(340, 0, accuracy=0.01)

21.707999999999696

# 2 Time calculation

**Question:** Does the trickle down njit compatibility make a large impact?

In [ ]:
def time_calculation()

In [163]:
velocity_function(340, 0, POWER)*3.6

78.27839999999891

In [ ]:
def calculate_time_matrix(customers, blabla, gap_size)

# 3 Distance calculation (Test cases)

In [1]:
import pandas as pd

In [2]:
?pd.read_excel

In [2]:
from scipy import constants

In [3]:
constants.gravitational_constant

6.67408e-11

In [44]:
np.zeros((2,2,3))

array([[[0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.]]])

In [73]:
t = 1
t is not None

True